In [24]:
import numpy as np

In [26]:

# Had a little help as always
# https://www.geeksforgeeks.org/dice-throw-problem/

# Here is the table of correct answers for 6 sided dice, to check accuracy
# https://wizardofodds.com/gambling/dice/2/
def find_ways(k,n,x):
    if x<1: return 0
    if n==1: return int(x<=k)#1 if true else 0
    num_ways = 0
    for i in range(1,k+1):
        num_ways += find_ways(k,n-1,x-i)
    return num_ways

In [27]:
find_ways(6,5,14)

540

In [88]:
'''
Dynamic Solution -- translated to numpy
'''
def findWays(k,n,x):
    # The extra row/col make indexing easier, but could be removed (it's not really too bad tho)
    table = np.zeros((n+1,x+1))
    
    if k < x:
        table[1][1:k+1] = 1
    else:
        table[1][1:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1

                
    print(table)
    return table[n][x]

    

In [106]:
findWays(6,1,6)

[[ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  1.  1.  1.  1.]]


1.0

In [112]:
def get_freq(k,n):
    x = n*k
    table = np.zeros((n+1,x+1))
    
    if k < x:
        table[1][:k+1] = 1
    else:
        table[1][:x+1] = 1
    
    
    for i in range(2,n+1):
        for j in range(2,x+1):
            l = 1
            while l<j and l<=k:
                table[i][j] += table[i-1][j-l]
                l+=1
    # print(table)
    return table[n][n:]

In [113]:
get_freq(6,2)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  5.,  4.,  3.,  2.,  1.])

In [114]:
def get_list_freq(dice):
    #setup
    #there are n dice, like last time
    n = dice.size
    # x needs to be the upper bound, which would be rolling k on all dice
    x = np.sum(dice)
    # init the table
    table = np.zeros((n+1,x+1))
    
    if np.max(dice) < x:
        table[1][:np.max(dice)+1] = 1
    else:
        table[1][:x+1] = 1
    
    i = 1
    for k in dice:
        # we need to handle each dice face independently
        #might need temporary bound for 2nd loop
        for j in range(2,x+1):
            l = 1
            while l<j and l <=k:
                table[i][j] += table[i-1][j-l]
                l+=1   
        i += 1
    # print(table)
    return table[n][n:]

In [119]:
print(get_list_freq(np.array([6,6,6])))
print(get_freq(6,3))

[  1.   3.   6.  10.  15.  21.  25.  27.  27.  25.  21.  15.  10.   6.   3.
   1.]
[  1.   3.   6.  10.  15.  21.  25.  27.  27.  25.  21.  15.  10.   6.   3.
   1.]


In [ ]:
def get_dict_freq(some_dict):
    